In [1]:
import random
from tabulate import tabulate
import plotly.express  as px
#import plotly.io as pio

In [2]:
def get_reward(gene,test = False):

    n = length_of_atom = 3

    primary_conflict = 0
    secondary_conflict = 0

    random = gene
    new_list = [random[i:i+n] for i in range(0, len(random), n)]

    table_student_group_1 = [] 
    table_student_group_2 = [] 
    table_student_group_3 = []

    table_class_1 = []
    table_class_2 = []
    table_class_3 = []
    chromosomes = []


    for j in range(0, len(new_list)):
        
        if (new_list[j][1]== 'a'):
            table_student_group_1.append(new_list[j])
        
        elif (new_list[j][1]== 'b'):
            table_student_group_2.append(new_list[j])
        
        elif (new_list[j][1]== 'c'):
            table_student_group_3.append(new_list[j])


    for j in range(0, len(new_list)):

        if (new_list[j][2]== '1'):
            table_class_1.append(new_list[j])

        elif (new_list[j][2]== '2'):
            table_class_2.append(new_list[j])

        elif (new_list[j][2]== '3'):
            table_class_3.append(new_list[j])


    for i in range(0, len(gene), 9):
        chromosome = gene[i:i+9]
        chromosomes.append(chromosome)
        if i % 3 == 0:
            primary_conflict += chromosome.count("G")
        group_count = {combo: 0 for combo in 'abc'}
        class_count = {combo: 0 for combo in '123'}

        for a in range(0,3):

            if chromosome[2+3*a] in '123':
                class_count[chromosome[2+3*a]] += 1

            if chromosome[1+3*a] in 'abc':
                group_count[chromosome[1+3*a]] += 1
        
        if  (max([a for a in group_count.values()])>1):
            primary_conflict+=1

        if  (max([a for a in class_count.values()])>1):
            primary_conflict+=1

    if len(table_student_group_1) != 18:primary_conflict += 1

    if len(table_student_group_2) != 18:primary_conflict += 1

    if len(table_student_group_3) != 18:primary_conflict += 1

    subject_repetitions_count = {combo: 0 for combo in 'ABCDEF'}


    for i in table_student_group_1:

        if i[0] in subject_repetitions_count:
            subject_repetitions_count[i[0]] += 1


    for subject_repetitions_key in subject_repetitions_count.keys():

        if subject_repetitions_count[subject_repetitions_key]!=2:
            secondary_conflict += abs(abs(subject_repetitions_count[subject_repetitions_key])-2)
            

    subject_repetitions_count = {combo: 0 for combo in 'ABCDEF'}


    for i in table_student_group_2:

        if i[0] in subject_repetitions_count:
            subject_repetitions_count[i[0]] += 1


    for subject_repetitions_key in subject_repetitions_count.keys():

        if subject_repetitions_count[subject_repetitions_key]!=2:
            secondary_conflict += abs(abs(subject_repetitions_count[subject_repetitions_key])-2)


    subject_repetitions_count = {combo: 0 for combo in 'ABCDEF'}


    for i in table_student_group_3:

        if i[0] in subject_repetitions_count:
            subject_repetitions_count[i[0]] += 1


    for subject_repetitions_key in subject_repetitions_count.keys():

        if subject_repetitions_count[subject_repetitions_key]!=2:
            secondary_conflict += abs(abs(subject_repetitions_count[subject_repetitions_key])-2) 

            
    reward = primary_conflict*-100 -50*secondary_conflict
    return reward

In [3]:
#random.seed(0)
prob = 0.499

In [4]:
def create_gene():

    possible_combinations = [f"{subject}{group}{classroom}" for subject in "ABCDEFG" for group in "abc" for classroom in "123"]
    a = ''

    for i in range (0,54) : 
        random_num = random.choice(possible_combinations)
        a = a + str(random_num)

    return a

In [5]:
def pick_an_atom_for_me(atom_1,atom_2):

    possible_combinations = [f"{subject}{group}{classroom}" for subject in "ABCDEFG" for group in "abc" for classroom in "123"]
    x = random.uniform(0, 1)

    if x <= prob:
        return atom_1

    elif x <= prob*2:
        return atom_2

    return random.choice(possible_combinations)

In [6]:
def give_me_my_child(parent_1,parent_2):

    n = size_of_atom = 3 
    child = []
    parent_1_separated_into_atoms = [parent_1[i:i+n] for i in range(0, len(parent_1), size_of_atom)]
    parent_2_separated_into_atoms = [parent_1[i:i+n] for i in range(0, len(parent_2), size_of_atom)]

    for i in range(len(parent_1_separated_into_atoms)):
        child.append(pick_an_atom_for_me(parent_1_separated_into_atoms[i],parent_2_separated_into_atoms[i]))

    return ''.join(child)

In [7]:
def sort_my_genes(unsorted_list):

    test_list = []
    sorted_list = []

    for i in unsorted_list:
        test_list.append((i,get_reward(i)))

    test_list.sort(key = lambda x: x[1],reverse=True)

    for i in test_list:
        sorted_list.append(i[0])

    return sorted_list

In [8]:
def genetic_algotithm(list_of_parent_genes):

    children_genes = []

    for j in range(3):
        random.shuffle(list_of_parent_genes)

        for i in range(0,len(list_of_parent_genes)-1):
            children_genes.append(give_me_my_child(list_of_parent_genes[i],list_of_parent_genes[i+1]))          
        
    children_genes = sort_my_genes(children_genes)
    return children_genes[0:500]

In [9]:
def get_my_average_reward(generation):

    total_reward = 0
    gen = generation[0:200]

    for i in gen:
        total_reward += get_reward(i)

    return total_reward/len(gen)

rewards_data = []

In [14]:
def run_my_algo():
    global prob
    repeat = 0
    count = 0
    current_reward = -80000
    ancestory = []
    gen_temp = []
    gen_temp2 = []

    for i in range(0,500):
        gen_temp.append(create_gene())

    gen_temp2 = gen_temp
    ancestory.append(get_my_average_reward(gen_temp))
    print(f"my average reward is {ancestory[0]} for gen {0}")

    i = -1
    while (True):
        i = i + 1
        gen_temp = genetic_algotithm(gen_temp)
        ancestory.append(get_my_average_reward(gen_temp))
        print(f"The average loss is {ancestory[i+1]} for gen {i+1} and probaility of {prob}")

        if current_reward == ancestory[i+1]:
            repeat += 1
        
        else:
            repeat = 0
            prob = 0.5
            current_reward = ancestory[i+1]

        if repeat > 2 : 
            count +=1
            prob -= 0.001
            repeat = 3

        if ancestory[i+1] == 0:
            break

        
    return ancestory,gen_temp

In [ ]:
all_rewards,reward5 = run_my_algo()

my average reward is -4943.25 for gen 0
The average loss is -4342.5 for gen 1 and probaility of 0.33099999999999985
The average loss is -4138.5 for gen 2 and probaility of 0.5
The average loss is -3960.5 for gen 3 and probaility of 0.5
The average loss is -3846.0 for gen 4 and probaility of 0.5
The average loss is -3789.25 for gen 5 and probaility of 0.5
The average loss is -3750.0 for gen 6 and probaility of 0.5
The average loss is -3750.0 for gen 7 and probaility of 0.5
The average loss is -3750.0 for gen 8 and probaility of 0.5
The average loss is -3750.0 for gen 9 and probaility of 0.5
The average loss is -3731.25 for gen 10 and probaility of 0.499
The average loss is -3693.75 for gen 11 and probaility of 0.5
The average loss is -3616.75 for gen 12 and probaility of 0.5
The average loss is -3552.75 for gen 13 and probaility of 0.5
The average loss is -3489.25 for gen 14 and probaility of 0.5
The average loss is -3400.0 for gen 15 and probaility of 0.5
The average loss is -3400.0 fo

In [ ]:
fig = px.line(x=list(range(len(all_rewards))), y=all_rewards, 
              labels={'x': 'Generation', 'y': 'Reward'}, title='Reward Progression') 

fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
print(reward5[0])
print(reward5[420])

In [ ]:
def clean_list(rewards):
    
    clean =[]
    
    for i in range(len(rewards)):
        if get_reward(rewards[i]) == 0:
            clean.append(rewards[i])

    return clean

c_list = clean_list(reward5)
len(c_list)

In [ ]:
def display(gene):

    random = gene
    
    courses = ["PHYS 101","MATH 101","MATH 103","CHEM 101","BIOL 103","COMP 102","Break",
                "MCSC 201","MATH 201","MATH 204","MATH 208","COMP 202","EEEG 202","Break",
                "MATH 302","MATH 304","MATH 326","MGTS 301","COMP 315","COMP 316","Break"]
    
    n = 3
    new_list = [random[i:i+n] for i in range(0, len(random), n)]

    days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    a = []
    b = []
    c = []
    
    for j in range(0, len(new_list)):
    
        x = new_list[j][1]
        y = new_list[j][0]
        i = 7 * (ord(x) - ord('a')) + (ord(y)- ord('A'))
    
        if ( x == 'a'):
            a.append(courses[i])
    
        if ( x == 'b'):
            b.append(courses[i])
    
        if ( x == 'c'):
            c.append(courses[i])

    t_1 = [[a[j + i*3] for i in range(6)] for j in range(3)]
    t_2 = [[b[j + i*3] for i in range(6)] for j in range(3)]
    t_3 = [[c[j + i*3] for i in range(6)] for j in range(3)]
    
    print()
    print("                       Routine for 1st SEMESTER")
    print(tabulate(t_1, headers = days, tablefmt="fancy_grid"))
    print()
    print("                       Routine for 3rd SEMESTER")
    print(tabulate(t_2, headers = days, tablefmt="fancy_grid"))
    print()
    print("                       Routine for 5th SEMESTER")
    print(tabulate(t_3, headers = days, tablefmt="fancy_grid"))

In [ ]:
display(reward5[0])

In [ ]:
display(reward5[420])

In [ ]:
len(reward5)